# Geoprocessing: Tools and Services
## by: Jeremy Oster

### Goals of this project:

#### 1.
   Develop flexible code that lets the user reclassify an attribute within a vector feature class according to flexible criteria

#### 2. 
   Package the code up into a Geoprocessing tool that can be used within ArcGIS Desktop's GUI, within ModelBuilder, or in ArcPy.

#### 3. 
   Experiment with new ways of putting the tool I develop out onto the internet as a service that could be used by other people on other computers from afar without downloading or installing the code locally.

#### Goal 1:
- Assume the user gives you a valid shapefile (_mytable_)
- Have the user specify that s/he wants to reclassify a specific field (_infield_)
- store the results of the reclassify in a new field by a name the user specifies (_outfield_)
- Take a user provided table (_reclasstable_) that tells you how to do the reclassification. Specifically, the table has columns named _lowerbound_, _upperbound_, and _value_.
- For every row in _mytable_, have the code look at the value of _infield_ and decide what to write in the _outfield_...
    To do this: 
    1. go throuh the _reclasstable_ to find the first row in _reclasstable_ where the value from _infield_ is greater than or equal to the _lowerbound_ and less-than-or-equal-to the _upperbound_
    2. if this row exists: then _outfield_ should be set to _value_
    3. if no row in the _reclasstable_ matches the current value in _infield_, then _outfield_ should be set to a special number called _notfoundvalue_, which the user has also specified.

In [5]:
# ------------------------ Setup ----------------------------- #
import os
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')
import arcpy
from arcpy import env
env.overwriteOutput = True

# ----------------------------------------------------------- #
# Fields to be variables (** = used in code below):

                    # mytable **
                    # infield **
                    # outfield **
                    # reclasstable **
                    # lowerbound **
                    # upperbound **
                    # value **
                    # notfoundvalue **

# ----------------------------------------------------------- #                    
                    
# Environment set locally for testing
env.workspace = r"F:\Geog458 Adv Digital Geog\Geoprocessing Tool\temp_"
                    
# -------- Start of the code for the Tool ------------------- #

def reclassTool(mytable, infield, outfield, reclasstable, notfoundvalue):
    # Checks for the input shapefile
    if arcpy.Exists(mytable) == True:
        # Checks for the reclasstable
        if arcpy.Exists(reclasstable) == True:

            # Makes the output field in mytable based on the infield
            arcpy.AddField_management(mytable, outfield, "TEXT", "50", "", "", "", "NULLABLE")

# ------------------------- Part 1 ----------------------------- #            
            
            cursor1 = arcpy.da.UpdateCursor(mytable, [infield, outfield])
            
            # Makes a list of the possible reclass values
            values = []
            
            for row1 in cursor1:
                
                 # Makes the cursor for the reclasstable from which I make my values[] list
                cursor2 = arcpy.da.SearchCursor(reclasstable, ["lowerbound", "upperbound", "value"])
                
                # Goes through each row in reclasstable to find the matching value for current row
                for row2 in cursor2:

                    # if value is not in values, add it to values
                    if row2[2] not in values:
                        values.append(row2[2])

                del row2
                del cursor2
                
                # for debugging
                # by this line, values should be equal to (for our sample) [1, 2, 3, 4]
                # print values        

# ------------------------ Part 2 ----------------------------- #            

                # Makes cursor2 again so that we can test the lowerbound and upperbound values
                cursor2 = arcpy.da.SearchCursor(reclasstable, ["lowerbound", "upperbound", "value"])
                
                # Goes through each row in reclasstable *again* (for this table it runs 4 times, should find only 1 match)
                for row2 in cursor2:

                    # If infield value is greater than or equal to lowerbound 
                    # and lower than or equal to upperbound in any of the rows in reclasstable:
                    if row1[0] >= row2[0] and row1[0] <= row2[1]:
                        # ...then Oufield is equal to current matching value in reclasstable
                        row1[1] = row2[2]
                    else:
                        pass

                # for debugging:
                # check to see if at some point row1[1] is assigned correctly, but then overwritten later...
                # print row1[1]
                # print "---------"

                # if infield doesn't match any lower/upper-bound in reclasstable after loop, we need to test:
                # if the outfield at this point (after the loop) is still not equal to any of the possible 
                #      values in reclasstable                 
                if row1[1] not in values:

                    # ...then Outfield is equal to the notfoundvalue
                    row1[1] = notfoundvalue
                else:
                    pass

                # After choosing (in the example) between 1, 2, 3, 4, or 9999 for outfield, updates current row.
                cursor1.updateRow(row1)

                # for debugging:
                # Checks what each row is being assigned to
                # print row1[1]
                # print "___________"
                
            # Deletes rows and cursors to remove any potential locks on the input files
            del row1
            del row2
            del cursor1
            del cursor2
            
            print "Reclass Finished!"
            
        # For when there's no reclasstable:
        else: 
            print "Input Reclass Table does not exist."
    # For when there's no input shapefile:
    else: 
        print "Input shapefile does not exist."

## Sample input conditions and ouputs

Given the files:
    King.shp   
        and
    ReclassTableExample.dbf
    
Relcass the field PopDens12 in King.shp according to ReclassTableExample.dbf

In [6]:
reclassTool(r"F:\Geog458 Adv Digital Geog\Geoprocessing Tool\temp_\King.shp", r"PopDens12", r"ReclaDen", 
            r"F:\Geog458 Adv Digital Geog\Geoprocessing Tool\temp_\ReclassTableExample.dbf", 9999)

# Currently works!


Reclass Finished!
